In [2]:
import os
import numpy as np
os.chdir('/local/home/xuchen/fast-snarf')
import torch
from tqdm import trange, tqdm
import yaml
from munch import Munch
from lib.utils.check_sign import check_sign
from lib.model.smpl import SMPLServer
from lib.model.network_ngp import ImplicitNetwork
from lib.model.fast_snarf import ForwardDeformer
from lib.model.helpers import masked_softmax
from lib.utils.meshing import generate_mesh
from lib.model.sample import PointInSpace
from pytorch3d.io.obj_io import load_obj,load_objs_as_meshes
import trimesh
from pysdf import SDF
import pandas
from pytorch3d.ops import sample_points_from_meshes

with_texture = True
def occ_func(x):
    batch_points = 200000000
    acc = []
    for pts_d_split in torch.split(x, batch_points, dim=1):
        
        pts_d_split = (pts_d_split + deformer.offset) * deformer.scale
        pts_d_split = (pts_d_split+1)/2
        
        occ = network(pts_d_split, {'smpl': smpl_params[:,7:-10]/np.pi})[...,:1]
    
        mask = torch.logical_or(pts_d_split.max(-1).values >= 1, pts_d_split.min(-1).values <= 0)
        occ[mask] = -100

        acc.append(occ)
    acc = torch.cat(acc,dim=1).reshape(-1, 1)
    return acc

def prepare_data(scan_verts, scan_faces):

    _, num_verts, num_dim = scan_verts.shape
    random_idx = torch.randint(0, num_verts, [1, int(1e8), 1], device=scan_verts.device)
    random_pts = torch.gather(scan_verts, 1, random_idx.expand(-1, -1, num_dim)).float()
    pts_d = sampler.get_points(random_pts)


    mesh = trimesh.Trimesh(vertices=scan_verts[0].cpu().numpy(), faces=scan_faces.cpu().numpy())
    sdf_obj = SDF(mesh.vertices, mesh.faces)
    sdf_gt = sdf_obj(pts_d[0].cpu().numpy())
    sdf_gt = torch.tensor(sdf_gt).cuda().float().unsqueeze(0)
    # sdf_gt = sdf_gt.clamp(-1,1)
    return sdf_gt, pts_d


# Prepare the data


smpl_params = torch.zeros(86).cuda().float()[None]
smpl_params[0,0] = 1

gender      = 'male'
betas       = smpl_params[:,-10:]
smpl_server = SMPLServer(gender=gender, betas=betas).cuda()
sampler = PointInSpace(global_sigma=1.8, local_sigma=0.01)


smpl_outputs = smpl_server.forward(smpl_params)
smpl_verts = smpl_outputs['smpl_verts']
smpl_verts_cano = smpl_server.verts_c
smpl_faces = torch.tensor(smpl_server.smpl.faces.astype('int')).cuda()
num_dim = 3


smpl_tfs = smpl_outputs['smpl_tfs']
cond = {'smpl': smpl_params[:,7:-10].cuda()/np.pi, 'smpl_params': smpl_params.cuda()}

sdf_gt_smpl, pts_d_smpl = prepare_data(smpl_verts_cano, smpl_faces)

# Canonical model
network = ImplicitNetwork(3, 1, 64, 3).cuda()

# Deformer
opt = Munch(yaml.load(open('config/deformer/fast_snarf.yaml'))['model']['deformer']['opt'])
opt.skinning_mode = 'preset'
opt.cvg = float(opt.cvg)
opt.dvg = float(opt.dvg)
deformer = ForwardDeformer(opt,smpl_server).cuda()

# Optimizer
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)

for iter in trange(1000):
    optimizer.zero_grad()

    random_idx = torch.randint(0, pts_d_smpl.shape[1], [1, int(1e5), 1], device=pts_d_smpl.device)
    pts_c = torch.gather(pts_d_smpl, 1, random_idx.expand(-1, -1, num_dim)).float()
    sdf_gt_cur = torch.gather(sdf_gt_smpl, 1, random_idx.squeeze(-1)).float()

    pts_c = ((pts_c + deformer.offset) * deformer.scale+1)/2
    occ_pd = network(pts_c, cond)[:,:,:1]

    loss = torch.nn.functional.l1_loss(occ_pd, sdf_gt_cur.unsqueeze(-1))
    loss.backward()
    optimizer.step()
    if iter%100==0:
        print(loss.item())



/local/home/xuchen/fast-snarf/lib/model/smpl.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.betas = torch.tensor(betas).float().cuda()
/tmp/ipykernel_1488962/710704671.py:83: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  opt = Munch(yaml.load(open('config/deformer/fast_snarf.yaml'))['model']['deformer']['opt'])
  5%|▍         | 46/1000 [00:00<00:02, 451.49it/s]

104.52366638183594


 17%|█▋        | 173/1000 [00:00<00:02, 379.59it/s]

104.15771484375


 25%|██▌       | 252/1000 [00:00<00:01, 386.13it/s]

...

 85%|████████▌ | 853/1000 [00:02<00:00, 394.45it/s]

103.87657928466797


 97%|█████████▋| 974/1000 [00:02<00:00, 394.76it/s]

104.16500091552734


100%|██████████| 1000/1000 [00:02<00:00, 391.32it/s]


In [8]:
from lib.model.helpers import create_voxel_grid
import time
import numpy as np
import torch
from skimage import measure
from lib.libmise import mise
import trimesh

res = 64
start = time.time()
grid = create_voxel_grid(res,res,res,device='cuda')

start = time.time()

pts_c, intermediates = deformer(grid, cond, smpl_tfs, eval_mode=True)
mask = intermediates['valid_ids']

num_batch, num_point, num_init, num_dim = pts_c.shape
pts_c = pts_c.reshape(num_batch, num_point * num_init, num_dim)

pts_c = ((pts_c + deformer.offset) * deformer.scale+1)/2
occ_pd = network(pts_c, cond, mask=mask.reshape(num_batch*num_point * num_init))[:,:,:1].reshape(num_batch, num_point, num_init)
occ_pd = masked_softmax(occ_pd, mask, dim=-1, mode='max', soft_blend=5)
print((time.time()-start)*1000)

value_grid = occ_pd.reshape(res,res,res).data.cpu().numpy()

start = time.time()
verts, faces, normals, values = measure.marching_cubes_lewiner(
                                            volume=value_grid,
                                            gradient_direction='ascent',
                                            level=min(0, value_grid.max()))

# verts = (verts / mesh_extractor.resolution - 0.5) * scale
# verts = verts * gt_scale + gt_center
meshexport = trimesh.Trimesh(verts, faces, normals, vertex_colors=values)
meshexport.export('test.obj')
print((time.time()-start)*1000)



/tmp/ipykernel_1488962/3137189700.py:29: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  verts, faces, normals, values = measure.marching_cubes_lewiner(
face_normals incorrect shape, ignoring!


4.273414611816406
18.13340187072754


In [ ]:
import math
import time

import numpy as np

import taichi as ti

ti.init(arch=ti.gpu)
res = 1280, 720
color_buffer = ti.Vector.field(3, dtype=ti.f32, shape=res)
sdf_field = ti.field(dtype=ti.f32, shape=(128,128,128))
max_ray_depth = 6
eps = 1e-4
inf = 1e10

fov = 0.23
dist_limit = 100

camera_pos = ti.Vector([0.0, 0.32, 3.7])
light_pos = [-1.5, 0.6, 0.3]
light_normal = [1.0, 0.0, 0.0]
light_radius = 2.0


@ti.func
def intersect_light(pos, d):
    light_loc = ti.Vector(light_pos)
    dot = -d.dot(ti.Vector(light_normal))
    dist = d.dot(light_loc - pos)
    dist_to_light = inf
    if dot > 0 and dist > 0:
        D = dist / dot
        dist_to_center = (light_loc - (pos + D * d)).norm_sqr()
        if dist_to_center < light_radius**2:
            dist_to_light = D
    return dist_to_light


@ti.func
def out_dir(n):
    u = ti.Vector([1.0, 0.0, 0.0])
    if abs(n[1]) < 1 - eps:
        u = n.cross(ti.Vector([0.0, 1.0, 0.0])).normalized()
    v = n.cross(u)
    phi = 2 * math.pi * ti.random()
    ay = ti.sqrt(ti.random())
    ax = ti.sqrt(1 - ay**2)
    return ax * (ti.cos(phi) * u + ti.sin(phi) * v) + ay * n


@ti.func
def make_nested(f):
    f = f * 40
    i = int(f)
    if f < 0:
        if i % 2 == 1:
            f -= ti.floor(f)
        else:
            f = ti.floor(f) + 1 - f
    f = (f - 0.2) / 40
    return f


# https://www.iquilezles.org/www/articles/distfunctions/distfunctions.htm
@ti.func
def sdf(o):
    wall = ti.min(o[1] + 0.1, o[2] + 0.4)

    if ti.min(o) >= 0 and ti.max(o) < 1:
        return ti.min(wall, sdf_field[ti.cast(o*128,ti.int32)])
    
    # wall = ti.min(o[1] + 0.1, o[2] + 0.4)
    # sphere = (o - ti.Vector([0.0, 0.35, 0.0])).norm() - 0.36

    # q = ti.abs(o - ti.Vector([0.8, 0.3, 0])) - ti.Vector([0.3, 0.3, 0.3])
    # box = ti.Vector([ti.max(0, q[0]),
    #                  ti.max(0, q[1]),
    #                  ti.max(0, q[2])]).norm() + ti.min(q.max(), 0)

    # O = o - ti.Vector([-0.8, 0.3, 0])
    # d = ti.Vector([ti.Vector([O[0], O[2]]).norm() - 0.3, abs(O[1]) - 0.3])
    # cylinder = ti.min(d.max(), 0.0) + ti.Vector(
    #     [ti.max(0, d[0]), ti.max(0, d[1])]).norm()

    # geometry = make_nested(ti.min(sphere, box, cylinder))
    # geometry = ti.max(geometry, -(0.32 - (o[1] * 0.6 + o[2] * 0.8)))
    return wall #ti.min(wall, geometry)


@ti.func
def ray_march(p, d):
    j = 0
    dist = 0.0
    while j < 100 and sdf(p + dist * d) > 1e-6 and dist < inf:
        dist += sdf(p + dist * d)
        j += 1
    return ti.min(inf, dist)


@ti.func
def sdf_normal(p):
    d = 1e-3
    n = ti.Vector([0.0, 0.0, 0.0])
    sdf_center = sdf(p)
    for i in ti.static(range(3)):
        inc = p
        inc[i] += d
        n[i] = (1 / d) * (sdf(inc) - sdf_center)
    return n.normalized()


@ti.func
def next_hit(pos, d):
    closest, normal, c = inf, ti.Vector.zero(ti.f32,
                                             3), ti.Vector.zero(ti.f32, 3)
    ray_march_dist = ray_march(pos, d)
    if ray_march_dist < dist_limit and ray_march_dist < closest:
        closest = ray_march_dist
        normal = sdf_normal(pos + d * closest)
        hit_pos = pos + d * closest
        t = int((hit_pos[0] + 10) * 1.1 + 0.5) % 3
        c = ti.Vector(
            [0.4 + 0.3 * (t == 0), 0.4 + 0.2 * (t == 1), 0.4 + 0.3 * (t == 2)])
    return closest, normal, c


@ti.kernel
def fill_sdf():
    for x,y,z in sdf_field:
        o = ti.Vector([x,y,z])/128.
        sphere = (o - ti.Vector([0.0, 0.35, 0.0])).norm() - 0.36

        q = ti.abs(o - ti.Vector([0.8, 0.3, 0])) - ti.Vector([0.3, 0.3, 0.3])
        box = ti.Vector([ti.max(0, q[0]),
                         ti.max(0, q[1]),
                         ti.max(0, q[2])]).norm() + ti.min(q.max(), 0)

        O = o - ti.Vector([-0.8, 0.3, 0])
        d = ti.Vector([ti.Vector([O[0], O[2]]).norm() - 0.3, abs(O[1]) - 0.3])
        cylinder = ti.min(d.max(), 0.0) + ti.Vector(
            [ti.max(0, d[0]), ti.max(0, d[1])]).norm()

        geometry = make_nested(ti.min(sphere, box, cylinder))
        geometry = ti.max(geometry, -(0.32 - (o[1] * 0.6 + o[2] * 0.8)))

        sdf_field[x,y,z] = geometry

@ti.kernel
def render():
    for u, v in color_buffer:
        aspect_ratio = res[0] / res[1]
        pos = camera_pos
        d = ti.Vector([
            (2 * fov * (u + ti.random()) / res[1] - fov * aspect_ratio - 1e-5),
            2 * fov * (v + ti.random()) / res[1] - fov - 1e-5, -1.0
        ])
        d = d.normalized()

        throughput = ti.Vector([1.0, 1.0, 1.0])

        depth = 0
        hit_light = 0.00

        while depth < max_ray_depth:
            closest, normal, c = next_hit(pos, d)
            depth += 1
            dist_to_light = intersect_light(pos, d)
            if dist_to_light < closest:
                hit_light = 1
                depth = max_ray_depth
            else:
                hit_pos = pos + closest * d
                if normal.norm_sqr() != 0:
                    d = out_dir(normal)
                    pos = hit_pos + 1e-4 * d
                    throughput *= c
                else:
                    depth = max_ray_depth
        color_buffer[u, v] += throughput * hit_light
        
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import imageio
import tqdm



frames = []
fill_sdf()
for frame in tqdm.trange(60*5):
  render()
  img = color_buffer.to_numpy() * (1 / ( 0+ 1))
  img = img / img.mean() * 0.24
  img = np.sqrt(img)

  frames.append(img)

imageio.mimwrite('mpm88.mp4', frames)

In [ ]:

from lib.libmise import mise

scale = 1.1  # Scale of the padded bbox regarding the tight one.

verts = smpl_verts.squeeze(0).data.cpu().numpy()
gt_bbox = np.stack([verts.min(axis=0), verts.max(axis=0)], axis=0)
gt_center = (gt_bbox[0] + gt_bbox[1]) * 0.5
gt_scale = (gt_bbox[1] - gt_bbox[0]).max()

mesh_extractor = mise.MISE(32, 4, 0)
points = mesh_extractor.query()

# query occupancy grid
import time
with torch.no_grad():
    while points.shape[0] != 0:

        orig_points = points
        points = points.astype(np.float32)
        points = (points / mesh_extractor.resolution - 0.5) * scale
        points = points * gt_scale + gt_center
        points = torch.tensor(points,device=smpl_verts.device,dtype=torch.float32)
        print(points.shape)
        start = time.time()
        values = occ_func(points.unsqueeze(0))[:,0]
        print('occ',time.time()-start)

        start = time.time()
        values = values.data.cpu().numpy().astype(np.float64)
        print('cpu',time.time()-start)

        start = time.time()
        mesh_extractor.update(orig_points, values)
        print('update',time.time()-start)

        start = time.time()
        points = mesh_extractor.query()
        print('query',time.time()-start)

value_grid = mesh_extractor.to_dense()
print(value_grid.shape)


In [ ]:
 = 1.1  # Scale of the padded bbox regarding the tight one.

verts = smpl_verts.squeeze(0).data.cpu().numpy()
gt_bbox = np.stack([verts.min(axis=0), verts.max(axis=0)], axis=0)
gt_center = (gt_bbox[0] + gt_bbox[1]) * 0.5
gt_scale = (gt_bbox[1] - gt_bbox[0]).max()

mesh_extractor = mise.MISE(32, 4, 0)
points = mesh_extractor.query()

# query occupancy grid
import time
with torch.no_grad():
    while points.shape[0] != 0:

        orig_points = points
        points = points.astype(np.float32)
        points = (points / mesh_extractor.resolution - 0.5) * scale
        points = points * gt_scale + gt_center
        points = torch.tensor(points,device=smpl_verts.device,dtype=torch.float32)
        print(points.shape)
        start = time.time()
        values = occ_func(points.unsqueeze(0))[:,0]
        print('occ',time.time()-start)

        start = time.time()
        values = values.data.cpu().numpy().astype(np.float64)
        print('cpu',time.time()-start)

        start = time.time()
        mesh_extractor.update(orig_points, values)
        print('update',time.time()-start)

        start = time.time()
        points = mesh_extractor.query()
        print('query',time.time()-start)

value_grid = mesh_extractor.to_dense()
print(value_grid.shape)
